In [ ]:
import os
import numpy as np
from skimage import io

DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/X/preps/CH1/downsampled_cropped'
volume = []
for filename in sorted(os.listdir(DIR)):
    filepath = os.path.join(DIR, filename)
    volume.append(io.imread(filepath))
volume = np.array(volume)
volume = np.swapaxes(volume, 0, 2)

# Precomputed folder path
precompute_path = os.path.join(os.path.expanduser('~'), f'image_test')

# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (452, 452, 20000)

# Voxel offset
offset = (0, 0, 0)

# Layer type
layer_type = 'image'

In [ ]:
volume.shape

In [ ]:
import os
import sys
import numpy as np

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.SqlController import SqlController
from utils import get_segment_properties

DIR = os.path.join('./')
animal = 'MD589'
downsample_factor = 16
all_structures = False

volume_path = os.path.join(DIR, f'{animal}_annotations_down{downsample_factor}.npy')

# Numpy array whose shape should match (x, y, z, channel) or just (x, y, z) if it's single channel
with open(volume_path, 'rb') as file:
    volume = np.load(file)
print(volume.shape)

# Precomputed folder path
precompute_path = os.path.join(DIR, f'{animal}_annotations_down{downsample_factor}')

# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (14464, 14464, 20000)

# Voxel offset
offset = (0, 0, 0)

# Layer type
layer_type = 'segmentation'

# segmentation properties in the format of [(number1, label1), (number2, label2) ...]
# where number is an integer that is in the volume and label is a string that describes that segmenetation
segment_properties = get_segment_properties(all_known=all_structures)

In [ ]:
# Run this cell if you want to preview the numpy array before converting to precomputed format
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=resol), 
        voxel_offset=offset
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:
# https://github.com/seung-lab/cloud-volume
# Don't change all compress=False. It seems that Neuroglancer can only read with compress=False
from cloudvolume import CloudVolume

cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = volume.shape[3] if len(volume.shape) > 3 else 1,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = resol, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [64, 64, 64], # units are voxels
    volume_size = volume.shape[:3], # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

In [ ]:
import json

vol.info['segment_properties'] = 'names'
vol.commit_info()

segment_properties_path = os.path.join(precompute_path, 'names')
os.makedirs(segment_properties_path, exist_ok=True)

info = {
    "@type": "neuroglancer_segment_properties", 
    "inline": {
        "ids": [str(number) for number, label in segmentation_properties],
        "properties": [{
            "id": "label", 
            "description": "Name of structures",
            "type": "label",
            "values": [str(label) for number, label in segmentation_properties]
        }]
    }
}
with open(os.path.join(segment_properties_path, 'info'), 'w') as file:
    json.dump(info, file, indent=2)

In [ ]:
# https://github.com/seung-lab/igneous
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=True)
tasks = tc.create_downsampling_tasks(cloudpath, compress=False) # Downsample the volumes 
tq.insert(tasks)
tq.execute()

In [ ]:
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False) # The first phase of creating mesh
tq.insert(tasks)
tq.execute()

# It should be able to incoporated to above tasks, but it will give a weird bug. Don't know the reason
tasks = tc.create_mesh_manifest_tasks(cloudpath) # The second phase of creating mesh
tq.insert(tasks)
tq.execute()

In [ ]:
import tinybrain
from cloudvolume import CloudVolume

volume = full_brain_volume_annotated
factor = (2, 2, 1)
volumes = tinybrain.downsample_segmentation(volume, factor=factor, num_mips=2, sparse=False)
volumes.insert(0, volume)

path = 'file:///net/birdstore/Active_Atlas_Data/data_root/pipeline_data/test'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint32', # Channel images might be 'uint8'
    encoding = 'compressed_segmentation', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(path, info=info, compress=False, progress=False)

for mip, volume in enumerate(volumes):
    vol.add_scale(np.array(factor) ** mip)
    vol.commit_info()
    vol = CloudVolume(path, mip=mip, compress=False, progress=False)
    vol[:, :, :] = volume[:, :, :]
    
vol.info['segment_properties'] = 'names'
vol.commit_info()